-------------


# Perguntas de Negócio

## Objetivo Específico:

- Demonstrar habilidades avançadas em SQL e análise de dados, utilizando Python para estabelecer conexão com o banco de dados através do pyodbc e realizar consultas complexas utilizando tanto pandasql quanto operações diretas de SQL. O projeto visa evidenciar a capacidade de manipular e analisar grandes conjuntos de dados de forma eficiente, utilizando ferramentas tecnológicas atuais para extrair insights valiosos que suportem decisões baseadas em dados. A utilização do pyodbc permite uma conexão robusta e segura ao banco de dados SQL Server, enquanto o pandasql oferece uma maneira flexível e intuitiva de executar consultas SQL diretamente em DataFrames do pandas, facilitando análises complexas e a integração de processos de análise de dados dentro do ambiente Python.

### Objetivo do Projeto:
- Avaliar as tendências de acidentes em rodovias brasileiras ao longo de três anos, identificando padrões significativos e variáveis impactantes para desenvolver estratégias eficazes de redução de acidentes e melhorar a segurança rodoviária.

-----------------

In [6]:
from sqlalchemy import create_engine
import urllib
import pyodbc
import pandas as pd 
import pandasql as ps

def sql_query(query):
    return ps.sqldf(query, globals())

In [5]:
df = pd.read_csv(r'C:\Users\tiago\df_prf_oco_01.csv')

In [7]:


# Configurações da conexão
server = 'LAPTOP-HHOA8PKM\SQLEXPRESS'  
database = 'ContosoRetailDW'
driver = '{ODBC Driver 17 for SQL Server}'  

# String de conexão usando autenticação do Windows
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'

# Conectar ao banco de dados
conn = pyodbc.connect(conn_str)
print("Conectado com sucesso ao SQL Server usando a autenticação do Windows")


Conectado com sucesso ao SQL Server usando a autenticação do Windows


In [8]:

params = urllib.parse.quote_plus(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=LAPTOP-HHOA8PKM\SQLEXPRESS;"
    r"DATABASE=ContosoRetailDW;"
    r"Trusted_Connection=yes;"
)

conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(conn_str)

In [9]:
df.to_sql(
    name='MediaDeVitimasAcidentes',  # Nome da tabela no SQL Server
    con=engine,  # Objeto de conexão SQLAlchemy
    if_exists='replace',  # Substituir se a tabela já existir
    index=False,  # Não incluir o índice do DataFrame como uma coluna na tabela
    chunksize=500  # Inserir dados em lotes para melhorar a eficiência
)


-287

----------------------------------

#### Qual é o número total de acidentes por estado em cada ano?

In [10]:
query1 = """
SELECT uf, YEAR(data_inversa) AS ano, COUNT(*) AS total_acidentes
FROM MediaDeVitimasAcidentes
GROUP BY uf, YEAR(data_inversa)
order by total_acidentes desc
"""
df1 = pd.read_sql_query(query1, engine)
print(df1)

    uf   ano  total_acidentes
0   MG  2023             9000
1   MG  2022             8295
2   SC  2023             7799
3   SC  2022             7593
4   PR  2022             7393
..  ..   ...              ...
76  SE  2024               84
77  AC  2024               35
78  AP  2024               20
79  RR  2024               19
80  AM  2024               13

[81 rows x 3 columns]



#### Variação Anual por Estado:

- **Minas Gerais (MG)** e **Santa Catarina (SC)** apresentam números altos de acidentes, com MG mostrando um aumento de acidentes de 2022 para 2023. Isso sugere uma possível deterioração nas condições de trânsito ou aumento no volume de tráfego.
- Por outro lado, estados com menor número de acidentes, como **Acre (AC)**, **Amapá (AP)**, **Roraima (RR)**, e **Amazonas (AM)** em 2024, podem indicar melhores condições de trânsito ou menor volume de veículos.




------------


#### Quantos acidentes resultaram em mortes por ano em cada estado

In [11]:
query2 = """
SELECT uf, YEAR(data_inversa) AS year, COUNT(*) AS acidentes_com_mortes
FROM MediaDeVitimasAcidentes
WHERE mortos > 0
GROUP BY uf, YEAR(data_inversa)
order by acidentes_com_mortes desc
"""
df2 = pd.read_sql_query(query2, engine)
print(df2)

    uf  year  acidentes_com_mortes
0   MG  2023                   609
1   MG  2022                   591
2   PR  2022                   495
3   PR  2023                   486
4   BA  2023                   454
..  ..   ...                   ...
76  RR  2024                     4
77  DF  2024                     3
78  AM  2024                     2
79  AP  2024                     1
80  AC  2024                     1

[81 rows x 3 columns]


#### Distribuição de Fatalidades:

- Minas Gerais (MG) e Paraná (PR) mostram números elevados de fatalidades em 2022 e 2023, indicando uma persistente gravidade dos acidentes nesses estados. Isso requer atenção especial para medidas de segurança e infraestrutura.
- A diminuição marginal nas fatalidades no Paraná de 2022 para 2023 (495 para 486) pode indicar um leve sucesso nas intervenções de segurança realizadas, mas ainda há muito espaço para melhorias.
- Estados como Acre (AC), Amapá (AP), e Amazonas (AM) têm números muito baixos de fatalidades em 2024, o que pode refletir melhorias significativas ou simplesmente um volume menor de tráfego.

--------------------------


###  Qual é a média de vítimas por acidente em cada tipo de acidente?

In [18]:
query3 = """
SELECT tipo_acidente, AVG(feridos) AS media_vitimas
FROM df
GROUP BY tipo_acidente
"""
result3 = sql_query(query3)
print(result3)   


                     tipo_acidente  media_vitimas
0          Atropelamento de Animal       1.071577
1        Atropelamento de Pedestre       0.947744
2                      Capotamento       1.304032
3               Colisão com objeto       0.967033
4                  Colisão frontal       1.636816
5    Colisão lateral mesmo sentido       1.088042
6   Colisão lateral sentido oposto       1.244093
7              Colisão transversal       1.288628
8                 Colisão traseira       1.128140
9            Derramamento de carga       0.439560
10                   Engavetamento       1.269373
11                Eventos atípicos       0.906344
12                        Incêndio       0.048692
13    Queda de ocupante de veículo       1.162447
14       Saída de leito carroçável       1.162953
15                      Tombamento       1.126654


#### Acidentes Mais Graves:

- Colisão frontal tem a média mais alta de vítimas por acidente (1.636816), o que é esperado, dado que colisões frontais frequentemente ocorrem em alta velocidade e envolvem impactos diretos.
- Capotamento e Colisão transversal também apresentam médias altas de vítimas, sugerindo que esses tipos de acidentes são particularmente perigosos.

--------------------------------------

### Quais são as horas do dia e os dias da semana com maior número de acidentes?

In [13]:
query4 = """
SELECT dia_semana, horario, COUNT(*) AS total_acidentes
FROM MediaDeVitimasAcidentes
GROUP BY dia_semana, horario
ORDER BY total_acidentes DESC
"""
df4 = pd.read_sql_query(query4, engine)
print(df4)

        dia_semana   horario  total_acidentes
0          domingo  19:00:00              401
1          domingo  20:00:00              347
2          domingo  18:30:00              324
3           sábado  19:00:00              314
4          domingo  18:00:00              310
...            ...       ...              ...
7864       domingo  15:03:00                1
7865  quarta-feira  15:03:00                1
7866  quinta-feira  14:57:00                1
7867  quarta-feira  14:59:00                1
7868   sexta-feira  14:59:00                1

[7869 rows x 3 columns]


#### Picos de Acidentes aos Domingos:

- O maior número de acidentes ocorre no final da tarde e no início da noite de domingo, especialmente às 19:00, 20:00 e 18:30. Isso pode estar relacionado ao retorno de viagens de fim de semana, quando há um aumento do fluxo nas estradas e possivelmente mais cansaço entre os motoristas.

#### Acidentes Significativos aos Sábados à Noite:

- Sábados também mostram um pico significativo de acidentes às 19:00. Isso sugere que as atividades de lazer e sociais do fim de semana podem contribuir para o aumento de acidentes durante esse período.

#### Distribuição ao Longo da Semana:

- Os acidentes parecem ser mais concentrados nos fins de semana, mas há incidências ao longo de toda a semana, embora em menor quantidade. A variação durante os dias úteis pode estar relacionada aos horários de pico de tráfego associados ao deslocamento para o trabalho e escola.

-----------------------

###  Qual rodovia teve o maior número de acidentes nos últimos três anos e qual foi a causa mais comum desses acidentes?

In [14]:
query6 = """
SELECT TOP 1 br, COUNT(*) AS total_acidentes,
       (SELECT TOP 1 causa_acidente FROM MediaDeVitimasAcidentes sub WHERE sub.br = MediaDeVitimasAcidentes.br GROUP BY causa_acidente ORDER BY COUNT(*) DESC) AS causa_mais_comum
FROM MediaDeVitimasAcidentes
WHERE YEAR(data_inversa) BETWEEN YEAR(GETDATE())-3 AND YEAR(GETDATE())
GROUP BY br
ORDER BY total_acidentes DESC
"""
df6 = pd.read_sql_query(query6, engine)
print(df6)

      br  total_acidentes                          causa_mais_comum
0  101.0            24526  Reação tardia ou ineficiente do condutor


#### Alta Incidência na BR-101:

- A rodovia BR-101 é uma das principais rodovias do Brasil, atravessando vários estados. Sua alta incidência de acidentes pode estar relacionada ao grande volume de tráfego, incluindo tráfego pesado de caminhões e longas distâncias percorridas pelos motoristas.
Causa Comum - Reação Tardia do Condutor:

- A causa mais comum de acidentes ser "Reação tardia ou ineficiente do condutor" sugere problemas com a percepção, o julgamento ou as ações dos motoristas. Isso pode incluir distração ao dirigir, fadiga, ou falha em ajustar a condução às condições da rodovia

----

### Qual é a relação entre condições meteorológicas e frequência de acidentes? 

In [15]:
query_clima = """
SELECT condicao_metereologica AS condicao_meteorologica, COUNT(*) AS total_acidentes
FROM MediaDeVitimasAcidentes
GROUP BY condicao_metereologica
ORDER BY total_acidentes DESC
"""
df_clima = pd.read_sql_query(query_clima, engine)
print(df_clima)

  condicao_meteorologica  total_acidentes
0              Céu Claro            88444
1                Nublado            22629
2                  Chuva            15366
3                    Sol             8328
4         Garoa/Chuvisco             5244
5               Ignorado             1864
6       Nevoeiro/Neblina             1187
7                  Vento              242
8                Granizo                7
9                   Neve                3


#### Predomínio de Acidentes em Céu Claro:

- Surpreendentemente, a maioria dos acidentes ocorre sob condições de céu claro (88,444 acidentes). Isso pode sugerir que os motoristas tendem a dirigir mais descuidadamente ou em velocidades mais altas quando as condições de condução são percebidas como ideais.
Impacto da Chuva e Nublado:

- As condições de chuva (15,366 acidentes) e céu nublado (22,629 acidentes) também estão associadas a um número significativo de acidentes. Isso indica que a visibilidade reduzida e as estradas escorregadias aumentam o risco de acidentes.
Condições Extremas Menos Frequentes:

- Condições extremas como nevoeiro/neblina, vento, granizo e neve têm números relativamente baixos de acidentes, o que pode ser atribuído à menor frequência dessas condições ou a uma maior cautela dos motoristas nessas condições.

-------------------------

### Quais municípios apresentam os maiores números de acidentes e qual a evolução desses números ao longo dos anos?

In [16]:
query_municipios = """
SELECT municipio, YEAR(data_inversa) AS ano, COUNT(*) AS total_acidentes
FROM MediaDeVitimasAcidentes
GROUP BY municipio, YEAR(data_inversa)
ORDER BY total_acidentes DESC
"""
df_municipios = pd.read_sql_query(query_municipios, engine)
print(df_municipios)

                    municipio   ano  total_acidentes
0                    BRASILIA  2023              977
1                    BRASILIA  2022              921
2                    CURITIBA  2022              900
3                   GUARULHOS  2023              830
4                   GUARULHOS  2022              759
...                       ...   ...              ...
5029                SAO BORJA  2024                1
5030             SAO BENTINHO  2024                1
5031  SANTO ANTONIO DE LISBOA  2024                1
5032     SANTANA DA BOA VISTA  2024                1
5033       SANTANA DO IPANEMA  2024                1

[5034 rows x 3 columns]


#### Altas Taxas de Acidentes em Grandes Centros Urbanos:

- Brasília e Guarulhos mostram um grande número de acidentes, com Brasília liderando em 2023 e 2022. A presença constante dessas grandes cidades na lista indica volumes de tráfego elevados e possíveis questões de infraestrutura ou de comportamento dos motoristas.
Evolução Anual dos Acidentes:

- Brasília mostrou um aumento nos acidentes de 921 em 2022 para 977 em 2023, indicando uma tendência preocupante que requer atenção.
Guarulhos também mostrou aumento de acidentes de 2022 para 2023, subindo de 759 para 830.
Distribuição Geográfica dos Acidentes:

- Cidades menores, como São Borja e Santo Antônio de Lisboa, têm incidências muito baixas de acidentes, o que pode refletir menores volumes de tráfego ou possivelmente melhores condições de condução.
Recomendações:

-------------------------------------

### Há uma tendência de redução ou aumento no número de acidentes ao longo dos anos em determinadas rodovias?

In [17]:
query_tendencia_rodovias = """
SELECT br, YEAR(data_inversa) AS ano, COUNT(*) AS total_acidentes
FROM MediaDeVitimasAcidentes
GROUP BY br, YEAR(data_inversa)
ORDER BY br, ano
"""
df_tendencia_rodovias = pd.read_sql_query(query_tendencia_rodovias, engine)
print(df_tendencia_rodovias)

        br   ano  total_acidentes
0      0.0  2022              199
1      0.0  2023              159
2      0.0  2024               29
3     10.0  2022              339
4     10.0  2023              462
..     ...   ...              ...
336  494.0  2023                1
337  495.0  2022               19
338  495.0  2023               41
339  495.0  2024                5
340  498.0  2023                1

[341 rows x 3 columns]


### Rodovia BR-0:

- 2022: 199 acidentes
- 2023: 159 acidentes
- 2024: 29 acidentes
-Observa-se uma tendência clara de redução no número de acidentes nesta rodovia ao longo dos anos, o que pode indicar que intervenções ou melhorias na segurança rodoviária têm sido eficazes.

- Rodovia BR-10:

- 2022: 339 acidentes
- 2023: 462 acidentes
- Há um aumento significativo no número de acidentes de 2022 para 2023, sugerindo que podem existir problemas crescentes nesta rodovia que necessitam de atenção imediata para prevenir futuros acidentes.

- Rodovia BR-495:

- 2022: 19 acidentes
- 2023: 41 acidentes
2-024: 5 acidentes
- Aumento de 2022 para 2023 seguido por uma redução drástica em 2024 pode indicar a implementação de medidas efetivas recentemente ou variações anormais que requerem investigação adicional.

---------------